## The NGNet
--*Neural G_Factor Net*--
Simple neural network used for the computation of the G_Factor, a numerical value defined by specific values of a graph built on the links of a page.

### Training Phase

The training process is quite standard and straight-forward: given the n G_features we want to directly predict the associated class.

### Employment Phase

The training model will be inserted in a wider model called X and utilized as a function for the computation of the G_Factor

In [225]:
!export CUDA_LAUNCH_BLOCKING=1

In [226]:
import torch
from torch.nn import Module
from torch import nn

In [227]:
class NGNet(Module):
    def __init__(self, num_features:int, hide_size:int, G_dim:int, n_class:int) -> None:
        super().__init__()
        

        self.encoder = nn.Sequential(
            nn.Linear(in_features=num_features, out_features=G_dim),
            
        )

        self.decoder = nn.Sequential(
            nn.Linear(in_features=G_dim, out_features=num_features),
            
        )
        self.classifier = nn.Sequential(
            nn.Linear(in_features=num_features, out_features=n_class)
        )
    def forward(self, x:torch.Tensor) -> torch.Tensor:
        x = self.encoder(x)
        x = self.decoder(x)
        x = self.classifier(x)
        return x

## Dataset

Load the dataset

In [228]:
from Train_Classificator import Train_Classificator
from PyDataset import PyDataset
from torch.utils.data import DataLoader
from pathlib import PosixPath

In [229]:
dataset = PyDataset(PosixPath('./dataset/validation.tsv'), ['G_num_cliques','G_mean_pr','G_nodes'],  target_label='label')
dataloader = DataLoader(dataset, batch_size=16)

## Network

In [230]:
network = NGNet(num_features=3, hide_size=3, G_dim=1, n_class=3)

## Train

In [231]:
trainer = Train_Classificator(network, torch.nn.CrossEntropyLoss(), torch.optim.Adam(params=network.parameters(), lr=0.01))

In [232]:
m = trainer.fit_and_get(dataloader, PosixPath('./NGNet/'), 10)

Epoch 10: 100%|██████████| 19/19 [00:00<00:00, 460.84it/s]


## Test

# X Net
--*full-no transformer model for Cultural Classification*--


In [ ]:
from sklearn.decomposition import SparsePCA
from sklearn.tree import DecisionTreeClassifier

In [ ]:
class XNet(nn.Module):
    def __init__(self, num_features:int, G_dim:int, G_weigths, num_comp:int):
        self._dimensional_red = SparsePCA(n_components=num_comp)
        self._NGN = NGNet(num_features=num_features, hide_size=0, G_dim=G_dim)
        self._NGN.classifier = nn.Identity()
        self.classifier = DecisionTreeClassifier()
        
    def fit(self, base_fe:np., G_fe:torch.Tensor) -> torch.Tensor:
        if self.training:
            
            fe_to_reduce = base_fe.numpy(force=True)

            reduced_fe = torch.Tensor(self._dimensional_red.fit(fe_to_reduce))
            G_factor = self._NGN(G_fe)


            
